# Notebook demonstrating how to access Historical Fire Perimeter data from geoserver

This notebook demonstrates how to access **Historical Fire Perimeter** data from <a href = "https://sdge.sdsc.edu/geoserver">geoserver </a>using the <a href = "https://docs.geoserver.org/stable/en/user/services/wfs/basics.html">Web Feature Service (WFS)</a> specification.

#### What dates are available?
- California: 1878 - 2022
- United States: 2006 - 2022

#### Where does the data come from?

The historical fire perimeter data are from the following sources:
- <a href = "https://frap.fire.ca.gov/">CAL FIRE FRAP Program</a>
- <a href = "https://data-nifc.opendata.arcgis.com">NIFC</a>
- <a href = "https://www.geomac.gov/">USGS GeoMAC</a>
- <a href = "https://www.blm.gov/">BLM</a>

**References:**
    
- WFS: https://docs.geoserver.org/latest/en/user/services/wfs/reference.html
- cql filter: https://docs.geoserver.org/stable/en/user/tutorials/cql/cql_tutorial.html

<h3>Notebook Overview:</h3>
<ol><a href='#all_attributes'><li>Display all attributes that can be queried</li></a>
    <a href='#dixie'><li>Query by specific fire (ex. Dixie fire)</li></a>
    <a href='#dates'><li>Query fire between particular dates and number of acres</li></a>
    <a href='#bbox'><li>Query fire between particular dates, number of acres, and bounding box,</li></a>
   

In [ ]:
# folium installation recommended for map plotting visualizations
# !pip install folium

In [1]:
import requests, json, folium
import geopandas as gpd

<a id='all_attributes'><h3>Display all attributes that can be queried</h3></a>

In [2]:
#let's take a look at the available attributes that are queryable in pandas

baseURL='https://firemap.sdsc.edu/geoserver/ows?'

selectionParameters = "service=WFS&version=2.0.0&request=DescribeFeatureType&typeName=WIFIRE:view_historical_fires&outputFormat=application%2Fjson" 

url = baseURL + selectionParameters

r = requests.get(url) 
data_dict = r.json()
d = data_dict['featureTypes'][0]['properties']
#d

df = gpd.GeoDataFrame(d).set_index('name').T
df

name,year,fire_name,agency,acres,mapmethod,inciwebid,alarm_date,cont_date,comments,complex_name,perimeter_timestamp,cause,geom
maxOccurs,1,1,1,1,1,1,1,1,1,1,1,1,1
minOccurs,0,0,0,0,0,0,0,0,0,0,0,0,0
nillable,True,True,True,True,True,True,True,True,True,True,True,True,True
type,xsd:number,xsd:string,xsd:string,xsd:number,xsd:string,xsd:string,xsd:date,xsd:date,xsd:string,xsd:string,xsd:date,xsd:string,gml:MultiPolygon
localType,number,string,string,number,string,string,date,date,string,string,date,string,MultiPolygon


### This is a typical query reponse in geojson output

In [3]:
baseURL='https://firemap.sdsc.edu/geoserver/ows?'

selectionParameters = "service=WFS&version=2.0.0&request=GetFeature&typeName=WIFIRE:view_historical_fires&count=1&outputFormat=application%2Fjson" 

url = baseURL + selectionParameters

r = requests.get(url) 
data_dict = r.json()
#data_dict

<a id='dixie'><h3>Query by specific fire name (ex. Dixie fire)</h3></a>

In [4]:
fire_name='DIXIE' 
#fire_name='DIXIE FIRE'.replace(' ','%20') # can't have spaces in api call

baseURL='https://firemap.sdsc.edu/geoserver/ows?'

selectionParameters = f"service=WFS&version=2.0.0&request=GetFeature&typeName=WIFIRE:view_historical_fires&cql_filter=fire_name='{fire_name}'&outputFormat=application%2Fjson"

url = baseURL + selectionParameters
print(url)
r = requests.get(url) 
data_dict = r.json()
#data_dict

# make list of dictionaries containing id, perimeter coordinates, timestamp, etc
rows = [{'id':entry['id'],'fire_name':entry['properties']['fire_name'],'acres':entry['properties']['acres'],'geometry':entry['geometry'],'perimeter_timestamp':entry['properties']['perimeter_timestamp'],'year':entry['properties']['year'],'cause':entry['properties']['cause'],'comments':entry['properties']['comments']} for entry in data_dict['features']]

df = gpd.GeoDataFrame.from_dict(rows, orient='columns')
df

# note - Query for Dixie is returning all fires with that name which includes several older fires. 
# Requester should include year and maybe a bbox to make sure they get a specific fire.

https://firemap.sdsc.edu/geoserver/ows?service=WFS&version=2.0.0&request=GetFeature&typeName=WIFIRE:view_historical_fires&cql_filter=fire_name='DIXIE'&outputFormat=application%2Fjson


,id,fire_name,acres,geometry,perimeter_timestamp,year,cause,comments
0,view_historical_fires.fid-4d4628be_17e556d6fe0...,DIXIE,8.639968,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2017-09-14Z,2017,None,WFDSS;GPS / Ground
1,view_historical_fires.fid-4d4628be_17e556d6fe0...,DIXIE,11.465830,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",None,2011,11,FRA/State DPA
2,view_historical_fires.fid-4d4628be_17e556d6fe0...,DIXIE,70.284797,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",None,1989,Natural,None
3,view_historical_fires.fid-4d4628be_17e556d6fe0...,DIXIE,NaN,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2021-07-20Z,2021,None,Feather River Canyon near Cresta Powerhouse
4,view_historical_fires.fid-4d4628be_17e556d6fe0...,DIXIE,NaN,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2021-07-20Z,2021,None,Feather River Canyon near Cresta Powerhouse
5,view_historical_fires.fid-4d4628be_17e556d6fe0...,DIXIE,NaN,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2021-07-16Z,2021,None,Feather River Canyon near Cresta Powerhouse
6,view_historical_fires.fid-4d4628be_17e556d6fe0...,DIXIE,8536.401599,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2021-07-17Z,2021,None,Feather River Canyon near Cresta Powerhouse
7,view_historical_fires.fid-4d4628be_17e556d6fe0...,DIXIE,NaN,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2021-07-20Z,2021,None,Feather River Canyon near Cresta Powerhouse
8,view_historical_fires.fid-4d4628be_17e556d6fe0...,DIXIE,NaN,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2021-07-20Z,2021,None,Feather River Canyon near Cresta Powerhouse
9,view_historical_fires.fid-4d4628be_17e556d6fe0...,DIXIE,NaN,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2021-07-22Z,2021,None,Feather River Canyon near Cresta Powerhouse


### Visualization fire perimeters

In [5]:
#Define coordinates of where we want to center our map
alert_coords = [39.871306,-121.389439]

#Create the map
my_map = folium.Map(location = alert_coords, zoom_start=7, width=500, height=250)

#GeoJSON alert marker overlays
folium.GeoJson(data_dict, name="geojson").add_to(my_map)   

#toggle layers
folium.LayerControl().add_to(my_map)  

#Display the map
#my_map

<a href = "https://nbviewer.org/github/BurnPro3D/data-api-notebooks/blob/main/access-historical-fire-perimeter-data.ipynb">view map using nbviewer</a>

<a id='dates'><h3>Query fire between particular dates and number of acres</h3></a>

In [6]:
start_year='2011'
end_year='2012'

min_acres = '19900'
max_acres = '20000'

baseURL='https://firemap.sdsc.edu/geoserver/ows?'

selectionParameters = f"service=WFS&version=2.0.0&request=GetFeature&typeName=WIFIRE:view_historical_fires&CQL_FILTER=year%20between%20{start_year}%20and%20{end_year}and%20acres%20between%20{min_acres}%20and%20{max_acres}&outputFormat=application%2Fjson"

url = baseURL + selectionParameters
print(url) 

r = requests.get(url) 
data_dict = r.json()
d = data_dict['features']
#d

# make list of dictionaries containing id, perimeter coordinates, timestamp, etc
rows = [{'id':entry['id'],'fire_name':entry['properties']['fire_name'],'acres':entry['properties']['acres'],'geometry':entry['geometry'],'perimeter_timestamp':entry['properties']['perimeter_timestamp'],'year':entry['properties']['year'],'cause':entry['properties']['cause'],'comments':entry['properties']['comments']} for entry in data_dict['features']]

df = gpd.GeoDataFrame.from_dict(rows, orient='columns')
df

https://firemap.sdsc.edu/geoserver/ows?service=WFS&version=2.0.0&request=GetFeature&typeName=WIFIRE:view_historical_fires&CQL_FILTER=year%20between%202011%20and%202012and%20acres%20between%2019900%20and%2020000&outputFormat=application%2Fjson


,id,fire_name,acres,geometry,perimeter_timestamp,year,cause,comments
0,view_historical_fires.fid-4d4628be_17e556d6fe0...,Horseshoe 2,19931.77,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2011-05-13Z,2011,None,IR heat perimeter
1,view_historical_fires.fid-4d4628be_17e556d6fe0...,Lion,19902.55,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2011-08-05Z,2011,None,None
2,view_historical_fires.fid-4d4628be_17e556d6fe0...,Lion,19982.96,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2011-08-06Z,2011,None,None
3,view_historical_fires.fid-4d4628be_17e556d6fe0...,Salt,19920.80,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2011-09-11Z,2011,None,None
4,view_historical_fires.fid-4d4628be_17e556d6fe0...,Jacks,19993.27,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2012-07-10Z,2012,None,WFDSS
5,view_historical_fires.fid-4d4628be_17e556d6fe0...,Wenatchee Complex,19999.60,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2012-10-06Z,2012,None,None


### Visualize perimeter on map

In [7]:
#Define coordinates of where we want to center our map
coords = [39.871306,-121.389439]

#Create the map
my_map = folium.Map(location = coords, zoom_start=4, width=500, height=250)

#GeoJSON marker overlays
folium.GeoJson(data_dict, name="geojson").add_to(my_map)   

#toggle layers
folium.LayerControl().add_to(my_map)  

#Display the map
my_map

<a href = "https://nbviewer.org/github/BurnPro3D/data-api-notebooks/blob/main/access-historical-fire-perimeter-data.ipynb">view map using nbviewer</a>

<a id='bbox'><h3>Query fire between particular dates, number of acres, and bbox</h3></a>

In [8]:
# define start/end year, min/max acres, and lat/long coordinates for the bounding box
start_year='2011'
end_year='2015'

min_acres = '10000'
max_acres = '20000'

min_Longitude, min_Latitude, max_Longitude, max_Latitude = -122.09596,39.66970,-120.336939,41.259914

baseURL='https://firemap.sdsc.edu/geoserver/ows?'

selectionParameters = f"service=WFS&version=2.0.0&request=GetFeature&typeName=WIFIRE:view_historical_fires&CQL_FILTER=year%20between%20{start_year}%20and%20{end_year}and%20acres%20between%20{min_acres}%20and%20{max_acres}and%28BBOX%28geom,{min_Longitude},%20{min_Latitude},%20{max_Longitude},%20{max_Latitude},%20%27EPSG:4269%27%20%29%29&outputFormat=application%2Fjson"

url = baseURL + selectionParameters
print(url) 

r = requests.get(url) 
data_dict = r.json()
d = data_dict['features']
#d

# make list of dictionaries containing id, perimeter coordinates, timestamp, etc
rows = [{'id':entry['id'],'fire_name':entry['properties']['fire_name'],'acres':entry['properties']['acres'],'geometry':entry['geometry'],'perimeter_timestamp':entry['properties']['perimeter_timestamp'],'year':entry['properties']['year'],'cause':entry['properties']['cause'],'comments':entry['properties']['comments']} for entry in data_dict['features']]

df = gpd.GeoDataFrame.from_dict(rows, orient='columns')
df

https://firemap.sdsc.edu/geoserver/ows?service=WFS&version=2.0.0&request=GetFeature&typeName=WIFIRE:view_historical_fires&CQL_FILTER=year%20between%202011%20and%202015and%20acres%20between%2010000%20and%2020000and%28BBOX%28geom,-122.09596,%2039.6697,%20-120.336939,%2041.259914,%20%27EPSG:4269%27%20%29%29&outputFormat=application%2Fjson


,id,fire_name,acres,geometry,perimeter_timestamp,year,cause,comments
0,view_historical_fires.fid-4d4628be_17e556d6fe0...,Bagley Complex,15101.230000,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2012-08-25Z,2012,None,None
1,view_historical_fires.fid-4d4628be_17e556d6fe0...,Likely,10061.890000,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2012-09-10Z,2012,None,FIMT
2,view_historical_fires.fid-4d4628be_17e556d6fe0...,Chips,18521.360000,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2012-08-09Z,2012,None,FIMT
3,view_historical_fires.fid-4d4628be_17e556d6fe0...,Reading,15497.600000,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2012-08-12Z,2012,None,None
4,view_historical_fires.fid-4d4628be_17e556d6fe0...,Reading,12533.430000,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2012-08-11Z,2012,None,None
5,view_historical_fires.fid-4d4628be_17e556d6fe0...,Chips,18228.050000,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2012-08-08Z,2012,None,FIMT
6,view_historical_fires.fid-4d4628be_17e556d6fe0...,Bagley Complex,11060.080000,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2012-08-24Z,2012,None,IR heat perimeter
7,view_historical_fires.fid-4d4628be_17e556d6fe0...,Bagley Complex,13711.390000,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2012-08-25Z,2012,None,IR heat perimeter
8,view_historical_fires.fid-4d4628be_17e556d6fe0...,Reading,19550.160000,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2012-08-13Z,2012,None,None
9,view_historical_fires.fid-4d4628be_17e556d6fe0...,Deer,11515.990000,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",2013-08-26Z,2013,None,IR heat perimeter


## visualize perimeters on map

In [9]:
#Define coordinates of where we want to center our map
coords = [40.66970,-122.09596]

#Create the map
my_map = folium.Map(location = coords, zoom_start=8, width=500, height=250)

#GeoJSON marker overlays
folium.GeoJson(data_dict, name="geojson").add_to(my_map)   

#toggle layers
folium.LayerControl().add_to(my_map)  

#Display the map
my_map

<a href = "https://nbviewer.org/github/BurnPro3D/data-api-notebooks/blob/main/access-historical-fire-perimeter-data.ipynb">view map using nbviewer</a>